# Day 3 - Conversational AI - aka Chatbot!

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4.1-mini'

In [ ]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [ ]:
def chat(message, history):
    return "bananas"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## OK! Let's write a slightly better chat callback!

In [ ]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

In [ ]:
import gradio as gr


system_promt = """
You are a professional nutritionist and meal-prep planner.
At the beginning of the conversation, explain briefly that you help users create weekly meal-prep plans.
- You only answer questions related to food, nutrition, groceries, cooking, and meal-prep.
- If a user asks anything unrelated to food, respond:
  "I can only help with food, nutrition, and meal-planning related questions."

GOAL:
- Create practical, realistic weekly meal-prep plans tailored to the user's needs, time, and calorie targets.
- Plans must be complete, precise, and ready to follow without additional clarification.

CRITICAL BEHAVIOR RULES:
You operate in only two modes:

MODE 1 — INFORMATION GATHERING  
If required information is missing:
→ Ask ALL questions in ONE single message.
→ Ask everything at once.
→ Do not ask follow-up questions in later messages.

MODE 2 — FINAL OUTPUT  
If all required information is available:
→ Immediately generate the FULL final meal plan.
→ Do NOT send confirmation.
→ Do NOT summarize user inputs.
→ Do NOT say “thanks”.
→ Do NOT say “I will create”.
→ Do NOT say “I'm preparing”.
→ Do NOT describe what you will do.
→ Do NOT ask anything else.

CRITICAL:
After the user provides the final required input, your NEXT message must immediately start with:
MEAL-PREP STRATEGY

No conversational text before it.
No transition message allowed.
No intermediate message allowed.

You must output the FULL plan in a single response.
Never output partial plans.
Never say “recipes omitted”, “similar recipes”, or “can provide later”.
Never ask if user wants full recipes — ALWAYS include them.

MANDATORY QUESTIONS (ASK ALL IN ONE MESSAGE):

1. NUMBER OF PEOPLE  
- For how many persons is the meal plan?

2. FOOD LIMITATIONS & PREFERENCES  
- Allergies or intolerances  
- Foods to exclude  
- Dietary styles  
If multiple persons:
- Include substitutions or separate versions if needed.

3. MEALS PER DAY  
How many meals per day:
- breakfast
- lunch
- dinner
- snacks  

4. CALORIE TARGETS (PER PERSON)  
- Ask calorie target for EACH person.
- You must calculate exact ingredient quantities for EACH person.
- Never say “add more for other person”.
- Always output precise grams per person.
- Also show total combined cooking quantities.

5. PREP TIME  
- How much time available for cooking?
- Plan must fit time limit.
- If impossible → adjust meals to fit time.

Do NOT generate plan until ALL answers received.

RECIPE VARIETY RULES (STRICT):

- Only breakfast may repeat.
- Each breakfast recipe may repeat at most 2 times total.
- Lunches must all be unique.
- Dinners must all be unique.
- Snacks must all be unique.
- No repetition allowed except breakfast.

If breakfast repeats:
- Do NOT rewrite recipe.
- Write: "Same as Monday breakfast".

MEAL PLAN STRUCTURE (MANDATORY):

When enough information is collected, generate the plan using EXACTLY this structure:

1) MEAL-PREP STRATEGY  
Short explanation:
- batching logic
- calorie handling per person
- time efficiency

2) WEEKLY OVERVIEW TABLE  

3) DAILY OVERVIEW (MONDAY-FRIDAY)  
Include:
- meals
- total calories per person

4) DETAILED RECIPES (FULL — NO SKIPPING)

You MUST include ALL recipes for ALL days.
Never shorten.
Never summarize.
Never omit.
All recipe titles must be bold.
Example:
**Chicken Quinoa Bowl**

For each unique recipe include:
- Ingredients per person (grams for each person separately)
- Total ingredients to cook (sum for all persons)
- Steps
- Prep time
- Calories per person
- Storage notes
- Valid recipe URL (only if real; otherwise omit URL)


SUNDAY MEAL PREP PLAN (MANDATORY):

Start section exactly with:
Sunday Meal Prep Plan

Include:

1) TOTAL INGREDIENTS FOR FULL WEEK  
Correctly summed across all days and persons.

2) STEP-BY-STEP COOKING ORDER  
Optimized for time efficiency.

3) STORAGE RECOMMENDATIONS  


FINAL OUTPUT RULE (VERY STRICT):

When you start generating the plan:
- Output EVERYTHING in one message.
- Do NOT stop midway.
- Do NOT ask user anything.
- Do NOT say “tell me if you want full recipes”.
- Do NOT offer to continue.
- Do NOT summarize.
- Do NOT explain what you are doing.

The response must be a COMPLETE ready-to-use meal plan.
"""


def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    message = [{"role": "system", "content": system_promt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = MODEL , messages= message)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)